In [1]:
import pandas as pd
import numpy as np 
import re

In [2]:
df = pd.read_csv(r'movies_profi.csv')

In [3]:
df.head()

,Month,Day,Title,Link,List,Year,Genres,Budget,Box office
0,1,2.0,The Woman in Black: Angel of Death,/wiki/The_Woman_in_Black:_Angel_of_Death,"['supernatural horror', '$15 million[6]', '$48...",2015.0,supernatural horror,$15 million[6],$48.9 million[1]
1,1,9.0,Taken 3,/wiki/Taken_3,"['-language French action-thriller', '$48 mill...",2015.0,-language French action-thriller,$48 million[2],$326.4 million[2]
2,1,16.0,Blackhat,/wiki/Blackhat_(film),"[' action thriller', '$70 million[3]', '$19.7 ...",2015.0,action thriller,$70 million[3],$19.7 million[3]
3,1,16.0,Spare Parts,/wiki/Spare_Parts_(2015_film),"['drama', '$4 million[2]', '$3.6 million[3]']",2015.0,drama,$4 million[2],$3.6 million[3]
4,1,16.0,The Wedding Ringer,/wiki/The_Wedding_Ringer,"[' buddy romantic comedy', '$23 million[4]', '...",2015.0,buddy romantic comedy,$23 million[4],$79.8 million[4]


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1032 entries, 0 to 1031
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Month       1032 non-null   int64  
 1   Day         1032 non-null   float64
 2   Title       1032 non-null   object 
 3   Link        1032 non-null   object 
 4   List        1032 non-null   object 
 5   Year        1032 non-null   float64
 6   Genres      1032 non-null   object 
 7   Budget      1032 non-null   object 
 8   Box office  1032 non-null   object 
dtypes: float64(2), int64(1), object(6)
memory usage: 72.7+ KB


### Cleaning the Budget col

In [5]:
df['Budget'] = df['Budget'].str.split('[').str.get(0)
df['Box office'] = df['Box office'].str.split('[').str.get(0)
df.tail()

,Month,Day,Title,Link,List,Year,Genres,Budget,Box office
1027,12,16.0,Avatar: The Way of Water,/wiki/Avatar:_The_Way_of_Water,"[' epic science fiction', '$350–460\xa0million...",2022.0,epic science fiction,$350–460 million,$2.32 billion
1028,12,21.0,Puss in Boots: The Last Wish,/wiki/Puss_in_Boots:_The_Last_Wish,"[' animated adventure comedy', '$90–110 millio...",2022.0,animated adventure comedy,$90–110 million,$484.6 million
1029,12,23.0,Babylon,/wiki/Babylon_(2022_film),"[' epic period black comedy-drama', '$78–80 mi...",2022.0,epic period black comedy-drama,$78–80 million,$63.4 million
1030,12,23.0,Whitney Houston: I Wanna Dance with Somebody,/wiki/Whitney_Houston:_I_Wanna_Dance_with_Some...,"[' biographical musical drama', '$45 million[1...",2022.0,biographical musical drama,$45 million,$59.8 million
1031,12,29.0,A Man Called Otto,/wiki/A_Man_Called_Otto,"[' comedy-drama', '$50 million', '$113.2 milli...",2022.0,comedy-drama,$50 million,$113.2 million


In [6]:
df.rename(columns={'Budget':'Budget2'}, inplace=True)

In [7]:
df['Budget2'] = df['Budget2'].str.replace(r'–', '-')

In [8]:
mask = df['Budget2'].str.find('-')
mask = mask != -1

In [9]:
(df.loc[mask, 'Budget2'].str.find('million') == -1).any()

False

In [10]:
def get_mean_budget(x):
    pattern = r'\$?([\d\.]*)-\$?([\d\.]*)'
    try:
        finds = re.findall(pattern, x)[0]
        return (float(finds[0]) + float(finds[1])) / 2
    except:
        pattern = r'\$?([\d\.]*)[\s\w]*-[\s\$]?([\d\.]*)'
        finds = re.findall(pattern, x)[0]
        return (float(finds[0]) + float(finds[1])) / 2      


df['Budget'] = df.loc[mask, 'Budget2'].apply(get_mean_budget)

In [11]:
mask = df['Budget'].isna()

In [12]:
(df.loc[mask, 'Budget2'].str.find('million') == -1).sum()

19

In [13]:
mask2 = (df.loc[mask, 'Budget2'].str.find('million') == -1)
df[mask].loc[mask2, 'Budget2']

82                 $100,000
84                 $100,000
200                $900,000
343                $500,000
356                 $80,000
402                $900,000
413                $880,000
659     $66 thousand (est.)
666                $586,443
671                £850,000
730                $500,000
734                $200,000
788               <$300,000
797                $250,000
871                $250,000
885                $950,000
902                $100,000
988                $500,000
1003               $250,000
Name: Budget2, dtype: object

In [14]:
df.loc[659, 'Budget'] = 0.066

In [15]:
def thousands_budget(x):
    pattern = r'.?\$(\d+),'
    try:
        finds = re.findall(pattern, x)[0]
        return float(finds) / 1000
    except:
        return 0.066
    
df.loc[(mask & mask2), 'Budget'] = df[mask].loc[mask2, 'Budget2'].apply(thousands_budget)

In [16]:
df.tail()

,Month,Day,Title,Link,List,Year,Genres,Budget2,Box office,Budget
1027,12,16.0,Avatar: The Way of Water,/wiki/Avatar:_The_Way_of_Water,"[' epic science fiction', '$350–460\xa0million...",2022.0,epic science fiction,$350-460 million,$2.32 billion,405.0
1028,12,21.0,Puss in Boots: The Last Wish,/wiki/Puss_in_Boots:_The_Last_Wish,"[' animated adventure comedy', '$90–110 millio...",2022.0,animated adventure comedy,$90-110 million,$484.6 million,100.0
1029,12,23.0,Babylon,/wiki/Babylon_(2022_film),"[' epic period black comedy-drama', '$78–80 mi...",2022.0,epic period black comedy-drama,$78-80 million,$63.4 million,79.0
1030,12,23.0,Whitney Houston: I Wanna Dance with Somebody,/wiki/Whitney_Houston:_I_Wanna_Dance_with_Some...,"[' biographical musical drama', '$45 million[1...",2022.0,biographical musical drama,$45 million,$59.8 million,NaN
1031,12,29.0,A Man Called Otto,/wiki/A_Man_Called_Otto,"[' comedy-drama', '$50 million', '$113.2 milli...",2022.0,comedy-drama,$50 million,$113.2 million,NaN


In [17]:
mask2 = (df.loc[mask, 'Budget2'].str.find('million') != -1)
df[mask].loc[mask2, 'Budget2'][:5]

0    $15 million
1    $48 million
2    $70 million
3     $4 million
4    $23 million
Name: Budget2, dtype: object

In [18]:
def millions_budget(x):
    pattern = r'[\$€£](\d+)'
    finds = re.findall(pattern, x)
    if len(finds) == 0:
        print('Error' + x)
    else:
        return float(finds[0])

df.loc[(mask & mask2), 'Budget'] = df[mask].loc[mask2, 'Budget2'].apply(millions_budget)

In [19]:
df.head()

,Month,Day,Title,Link,List,Year,Genres,Budget2,Box office,Budget
0,1,2.0,The Woman in Black: Angel of Death,/wiki/The_Woman_in_Black:_Angel_of_Death,"['supernatural horror', '$15 million[6]', '$48...",2015.0,supernatural horror,$15 million,$48.9 million,15.0
1,1,9.0,Taken 3,/wiki/Taken_3,"['-language French action-thriller', '$48 mill...",2015.0,-language French action-thriller,$48 million,$326.4 million,48.0
2,1,16.0,Blackhat,/wiki/Blackhat_(film),"[' action thriller', '$70 million[3]', '$19.7 ...",2015.0,action thriller,$70 million,$19.7 million,70.0
3,1,16.0,Spare Parts,/wiki/Spare_Parts_(2015_film),"['drama', '$4 million[2]', '$3.6 million[3]']",2015.0,drama,$4 million,$3.6 million,4.0
4,1,16.0,The Wedding Ringer,/wiki/The_Wedding_Ringer,"[' buddy romantic comedy', '$23 million[4]', '...",2015.0,buddy romantic comedy,$23 million,$79.8 million,23.0


In [20]:
df['Budget'].isna().sum()

0

### Cleaning Box office col

In [21]:
mask = (df['Box office'].str.find('million') == -1)
mask.sum()

118

In [22]:
df.loc[mask, 'Box office'][0:15]

6                                 $400,000
8                                 $408,918
18                                $139,436
21                                $145,427
39    $18,687,388 (Theatrical Performance)
44                          $1.515 billion
52                                $108,808
53                                $478,595
57                          $1.403 billion
58                                $774,780
68                                $933,847
73                          $1.671 billion
81                          $1.159 billion
84                                $935,900
85       $100,044 (Theatrical Performance)
Name: Box office, dtype: object

In [23]:
mask2 = (df['Box office'].str.find('billion') == -1)
mask2.sum()

1002

In [24]:
df.loc[(mask & mask2), 'Box office'][0:15]

6                                  $400,000
8                                  $408,918
18                                 $139,436
21                                 $145,427
39     $18,687,388 (Theatrical Performance)
52                                 $108,808
53                                 $478,595
58                                 $774,780
68                                 $933,847
84                                 $935,900
85        $100,044 (Theatrical Performance)
106                                $483,938
118                                $422,746
124                                 $90,777
136                                 $70,958
Name: Box office, dtype: object

In [25]:
df.rename(columns={'Box office':'Box office2'}, inplace=True)

In [26]:
pattern = r'\$([\d,\.]+)'

df.loc[(mask & mask2),'Box office'] = df.loc[(mask & mask2),
                          'Box office2'].str.findall(pattern).str.get(0).str.replace(',', '').astype('int32') / 1000000

In [27]:
df.loc[(mask & mask2),'Box office'][:10]

6      0.400000
8      0.408918
18     0.139436
21     0.145427
39    18.687388
52     0.108808
53     0.478595
58     0.774780
68     0.933847
84     0.935900
Name: Box office, dtype: float64

In [28]:
df['Box office'] = df['Box office'] / 1000000

In [29]:
mask = (df['Box office2'].str.find('million') != -1)
df.loc[mask,'Box office'] = df.loc[mask,
                          'Box office2'].str.findall(pattern).str.get(0).astype(float)

In [30]:
mask = (df['Box office2'].str.find('billion') != -1)
mask.sum()

30

In [31]:
df['Box office'].isna().sum()

30

In [34]:
df.loc[mask, 'Box office'] = df.loc[mask, 'Box office2'].str.findall(pattern).str.get(0).astype(float) * 1000

In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1032 entries, 0 to 1031
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Month        1032 non-null   int64  
 1   Day          1032 non-null   float64
 2   Title        1032 non-null   object 
 3   Link         1032 non-null   object 
 4   List         1032 non-null   object 
 5   Year         1032 non-null   float64
 6   Genres       1032 non-null   object 
 7   Budget2      1032 non-null   object 
 8   Box office2  1032 non-null   object 
 9   Budget       1032 non-null   float64
 10  Box office   1032 non-null   float64
dtypes: float64(4), int64(1), object(6)
memory usage: 88.8+ KB


### Cleaning Genres

In [43]:
df['Genres'] = df['Genres'].str.replace('-', ' ').str.strip().str.split(' ')

In [58]:
df['Genres'].apply(len).value_counts()

Genres
2     432
3     249
1     187
4     108
5      31
6       6
15      3
11      2
13      2
16      2
7       2
25      1
10      1
22      1
23      1
12      1
21      1
14      1
34      1
Name: count, dtype: int64

In [60]:
df.loc[(df['Genres'].apply(len) > 5), 'Genres']

156     [comedy, film, directed, by, Jason, Moore,, wr...
164     [adult, stop, motion, animated, psychological,...
193     [dystopian, science, fiction, action, film, di...
234     [fantasy, adventure, film, directed, and, co, ...
377     [action, crime, thriller, film, written, and, ...
394     [/, American, production, jukebox, musical, ro...
412            [adult, puppet, buddy, cop, crime, comedy]
519     [Christian, drama, film, directed, by, Roxann,...
521       [surrealist, neo, noir, black, comedy, mystery]
529     [comedy, film, directed, by, Zara, Hayes,, sta...
532     [family, comedy, drama, film, directed, by, Ga...
551         [direct, to, video, prison, action, thriller]
562     [comedy, drama, film, written, and, directed, ...
598     [drama, film, written, and, directed, by, Noah...
665     [spy, comedy, film, directed, by, Peter, Segal...
695     [horror, thriller, film, written, and, directe...
822     [supernatural, horror, film, directed, by, Sta...
831     [Ameri

In [64]:
df.loc[(df['Genres'].apply(len) == 4), 'Genres'][:10]

1           [language, French, action, thriller]
9          [animated, jukebox, musical, fantasy]
12            [found, footage, science, fiction]
16    [live, action/animated, adventure, comedy]
21            [musical, romantic, comedy, drama]
22                  [high, school, teen, comedy]
28         [dystopian, science, fiction, action]
32               [magic, realism, comedy, drama]
36         [dystopian, science, fiction, action]
41          [animated, science, fiction, comedy]
Name: Genres, dtype: object

In [65]:
df = df[(df['Genres'].apply(len) < 5)]

In [67]:
from sklearn.preprocessing import MultiLabelBinarizer

s = df['Genres']

mlb = MultiLabelBinarizer()

df_genres = pd.DataFrame(mlb.fit_transform(s),columns=mlb.classes_, index=df.index)
df_genres.head()

,,",",American,Belgian,Christian,Christmas,French,Gothic,II,Neo,...,the,thriller,tragicomedy,up,urban,vigilante,war,western,white,zombie
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [73]:
(df_genres.sum(axis=0) > 15).sum()

24

In [77]:
col_genres = df_genres.columns.to_series()
col_genres.head()

                      
,                    ,
American      American
Belgian        Belgian
Christian    Christian
dtype: object

In [83]:
col_to_keep = col_genres[(df_genres.sum(axis=0) > 10)].index.to_list()

In [84]:
df_genres = df_genres[col_to_keep]
df_genres.head()

,Christian,Western,action,adventure,animated,biographical,black,buddy,comedy,crime,...,psychological,romantic,science,slasher,sports,spy,superhero,supernatural,thriller,war
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,1,1,0,...,0,1,0,0,0,0,0,0,0,0


In [86]:
df = pd.concat([df, df_genres], axis=1)
df.head()

,Month,Day,Title,Link,List,Year,Genres,Budget2,Box office2,Budget,...,psychological,romantic,science,slasher,sports,spy,superhero,supernatural,thriller,war
0,1,2.0,The Woman in Black: Angel of Death,/wiki/The_Woman_in_Black:_Angel_of_Death,"['supernatural horror', '$15 million[6]', '$48...",2015.0,"[supernatural, horror]",$15 million,$48.9 million,15.0,...,0,0,0,0,0,0,0,1,0,0
1,1,9.0,Taken 3,/wiki/Taken_3,"['-language French action-thriller', '$48 mill...",2015.0,"[language, French, action, thriller]",$48 million,$326.4 million,48.0,...,0,0,0,0,0,0,0,0,1,0
2,1,16.0,Blackhat,/wiki/Blackhat_(film),"[' action thriller', '$70 million[3]', '$19.7 ...",2015.0,"[action, thriller]",$70 million,$19.7 million,70.0,...,0,0,0,0,0,0,0,0,1,0
3,1,16.0,Spare Parts,/wiki/Spare_Parts_(2015_film),"['drama', '$4 million[2]', '$3.6 million[3]']",2015.0,[drama],$4 million,$3.6 million,4.0,...,0,0,0,0,0,0,0,0,0,0
4,1,16.0,The Wedding Ringer,/wiki/The_Wedding_Ringer,"[' buddy romantic comedy', '$23 million[4]', '...",2015.0,"[buddy, romantic, comedy]",$23 million,$79.8 million,23.0,...,0,1,0,0,0,0,0,0,0,0


### Ordinate the dataframe

In [87]:
df['Profit'] = df['Box office'] - df['Budget']
df['Profit perc'] = df['Profit'] * 100/ df['Budget']
df[['Box office', 'Budget', 'Profit', 'Profit perc']][:5]

,Box office,Budget,Profit,Profit perc
0,48.9,15.0,33.9,226.000000
1,326.4,48.0,278.4,580.000000
2,19.7,70.0,-50.3,-71.857143
3,3.6,4.0,-0.4,-10.000000
4,79.8,23.0,56.8,246.956522


In [93]:
new_col = [
    'Title','Budget','Profit','Box office','Profit perc',
    'Year','Month','Day'
]

new_col = new_col + col_to_keep

In [96]:
df[new_col[1:]] = df[new_col[1:]].astype('float32')

In [99]:
df = df[new_col]
df[new_col[:10]].info()

<class 'pandas.core.frame.DataFrame'>
Index: 976 entries, 0 to 1031
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Title        976 non-null    object 
 1   Budget       976 non-null    float32
 2   Profit       976 non-null    float32
 3   Box office   976 non-null    float32
 4   Profit perc  976 non-null    float32
 5   Year         976 non-null    float32
 6   Month        976 non-null    float32
 7   Day          976 non-null    float32
 8   Christian    976 non-null    float32
 9   Western      976 non-null    float32
dtypes: float32(9), object(1)
memory usage: 49.6+ KB


## Exploration of Correlation

In [107]:
genres = col_to_keep

In [111]:
df[genres].corrwith(df['Budget']).sort_values(ascending=False)

superhero        0.381174
epic             0.218444
fantasy          0.204676
fiction          0.166939
science          0.162864
adventure        0.154471
animated         0.151830
action           0.148407
spy              0.091746
musical          0.062906
Western          0.016203
dystopian        0.003835
mystery         -0.000850
buddy           -0.006695
historical      -0.010917
war             -0.016866
heist           -0.021955
sports          -0.030053
neo             -0.030766
slasher         -0.064611
black           -0.079834
crime           -0.080266
biographical    -0.088123
Christian       -0.088997
psychological   -0.092628
thriller        -0.103590
supernatural    -0.113774
comedy          -0.119662
romantic        -0.128252
horror          -0.192514
drama           -0.254776
dtype: float64